In [1]:
#initial imports
from collections import Counter
from IPython.display import clear_output
import json
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import re
import seaborn as sns
import sys
import time
#BeautifulSoup
from bs4 import BeautifulSoup
#gensim
import bz2
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import multiprocessing
import gensim
from gensim import corpora
from gensim import models
import pickle
import pyLDAvis.gensim
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from numpy import array
from gensim.models import Phrases
#NLTK
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords # Import the stop word list
from nltk import word_tokenize
from nltk.corpus import wordnet as wn
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
#plotly
import plotly.plotly as py
from plotly.graph_objs import *
import plotly
import plotly.tools as tls
import plotly.graph_objs as go
import plotly.figure_factory as ff
from  plotly  import __version__
#plotly offline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
print(__version__) # requires version >= 1.9.0
init_notebook_mode(connected=True)
#sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_val_score
from sklearn import ensemble
from sklearn.decomposition import PCA
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neural_network
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble
from sklearn.decomposition import PCA
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn import neural_network
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier
from sklearn import linear_model
#SpaCy
import spacy
spacy.load('en')
from spacy.lang.en import English

2019-03-03 16:27:02,207 : INFO : 'pattern' package not found; tag filters are not available for English
/anaconda3/lib/python3.6/site-packages/ipywidgets/widgets/widget.py:281: DeprecationWarning:

Widget registration using a string name has been deprecated. Widget registration now uses a plain `@register` decorator.



3.1.0


In [2]:
df = pd.read_csv('~/Desktop/Final Capstone/jobs_db.csv')
pd.set_option('display.max_colwidth', -1)

In [3]:
df.head(10)

id  \
0  15   
1  16   
2  17   
3  18   
4  19   
5  20   
6  21   
7  22   
8  23   
9  24   

                                                                                                                                                                                                 title  \
0  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Scientist - Natural Language Processing</h3>                                                                     
1  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Scientist - Natural Language Processing</h3>                                                                     
2  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Scientist - Natural Language Processing</h3>                                                                     
3  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Natural Language Processing Engineer, New York City</h3>                                                                  
4  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Programmer I - Natural Language Processing</h3>                                                                  
5  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Siri - Natural Language Processing Scientist/Engineer</h3>                                                                
6  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">NLP and Machine Learning Graduate</h3>                                                                                    
7  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Data Scientist, Natural Language Processing (NLP), descriptive statistics, statistical modelling, machine learning</h3>   
8  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Software Engineer, Natural Language Processing /NLP</h3>                                                                  
9  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Natural Language Processing and Machine Learning Engineer</h3>                                                            

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

Turn our df into X, for later use with modelling, train_test_split and getting a y column.

In [4]:
X = df

## EDA and Data Cleaning

Since we web scraped, the text is in pretty raw format.  To get it into useable shape, I plan to get rid of the html code before tokenizing and lemmatizing it. I'll also check for duplicates, and remove any to keep the results as clean as possible.  From here, it will be easier to determine what words can be added as stop words, or, are meaningful to the analysis in some way.

In [5]:
#Let's check for unique job descriptions. 
#If we drop some duplicates, we'll run our cleaning on less data, which is faster.
X.nunique()

id                 7882
title              3464
job_description    4334
job_class          9   
dtype: int64

In [6]:
X.job_class.unique()

array(['Natural Language Processing', 'Text Mining',
       'Pattern Recognition', 'Machine Learning', 'Computer Vision',
       'Text Analytics', 'Apache Spark', 'Speech Recognition',
       'Neural Networks'], dtype=object)

In [7]:
X.groupby(X['job_class'])['job_description'].count().sort_values()

job_class
Pattern Recognition            792
Text Analytics                 795
Neural Networks                827
Computer Vision                829
Natural Language Processing    881
Apache Spark                   919
Machine Learning               934
Speech Recognition             944
Text Mining                    961
Name: job_description, dtype: int64

In [8]:
#It stands to reason that we would want to keep the unique job descriptions, because titles like "Data Scientist"
#are broadly used in the field and not necessarily problematic for the scope of this project.
X = X.drop_duplicates(['job_description'], keep = 'last')

In [9]:
#Let's check to make sure we dropped what we had intended.
X.nunique()

id                 4334
title              3431
job_description    4334
job_class          9   
dtype: int64

In [10]:
X.groupby(X['job_class'])['job_description'].count().sort_values()

job_class
Text Analytics                 171
Text Mining                    171
Pattern Recognition            321
Natural Language Processing    510
Machine Learning               573
Computer Vision                633
Apache Spark                   645
Speech Recognition             648
Neural Networks                662
Name: job_description, dtype: int64

The sum of our "female dominated fields is 1,500 which is 34.6% of our dataset. I'm going to drop rows that are Computer Vision job descriptions.  This should help even out the imbalance, and that particular field was the most "gender-balanced" out of the male-dominated job descriptions we have above.

In [11]:
#Dropping Computer Vision
X = X[X.job_class != 'Computer Vision']

In [12]:
X.reset_index(drop=True, inplace=True)

In [13]:
X.head(10)

id  \
0  17   
1  18   
2  19   
3  20   
4  21   
5  29   
6  37   
7  38   
8  44   
9  45   

                                                                                                                                                      title  \
0  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Scientist - Natural Language Processing</h3>                          
1  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Natural Language Processing Engineer, New York City</h3>                       
2  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Programmer I - Natural Language Processing</h3>                       
3  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Siri - Natural Language Processing Scientist/Engineer</h3>                     
4  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">NLP and Machine Learning Graduate</h3>                                         
5  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">AI Research - Natural Language Processing(NLP) Scientist</h3>                  
6  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Analytical Linguist, Google AI</h3>                                            
7  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">IBM Research AI Residency Program</h3>                                         
8  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">AI Research Engineer – Natural Language Processing</h3>                        
9  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Software Engineer: Machine Learning - Natural Language Processing (NLP)</h3>   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [14]:
X.groupby(X['job_class'])['job_description'].count().sort_values()

job_class
Text Analytics                 171
Text Mining                    171
Pattern Recognition            321
Natural Language Processing    510
Machine Learning               573
Apache Spark                   645
Speech Recognition             648
Neural Networks                662
Name: job_description, dtype: int64

Although we lost a large portion of the dataset due to duplicates, it's important to prioritize quality over quantity. I evaluated the dataset to ensure that there would not be a class imbalance problem.  Now, our female-dominated fields comprise 40% of the job descriptions, this is pretty good considering we're working with scraped data. So, we'll proceed with cleaning up the dataset, so we can put it into BoW and Tf-Idf and then run some models on it!

In [15]:
#A quick view of the dataset to see what we might need to clean.
X.head(10)

id  \
0  17   
1  18   
2  19   
3  20   
4  21   
5  29   
6  37   
7  38   
8  44   
9  45   

                                                                                                                                                      title  \
0  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Scientist - Natural Language Processing</h3>                          
1  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Natural Language Processing Engineer, New York City</h3>                       
2  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Programmer I - Natural Language Processing</h3>                       
3  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Siri - Natural Language Processing Scientist/Engineer</h3>                     
4  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">NLP and Machine Learning Graduate</h3>                                         
5  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">AI Research - Natural Language Processing(NLP) Scientist</h3>                  
6  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Analytical Linguist, Google AI</h3>                                            
7  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">IBM Research AI Residency Program</h3>                                         
8  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">AI Research Engineer – Natural Language Processing</h3>                        
9  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Software Engineer: Machine Learning - Natural Language Processing (NLP)</h3>   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

Lots of HTML script here, as expected. Let's get some of that cleaned up before we tokenize.

In [16]:
X['job_description'] = X['job_description'].replace(r'div class="jobsearch-JobComponent-description icl-u-xs-mt--md"><div></div><div><div><div><b>', '', regex=True).replace(r'</div', '', regex=True)

In [17]:
X['title'] = X['title'].replace(r'<h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">', '', regex=True).replace(r'</h3>', '', regex=True)

In [18]:
X.head(10)

id  \
0  17   
1  18   
2  19   
3  20   
4  21   
5  29   
6  37   
7  38   
8  44   
9  45   

                                                                     title  \
0  Research Scientist - Natural Language Processing                          
1  Natural Language Processing Engineer, New York City                       
2  Research Programmer I - Natural Language Processing                       
3  Siri - Natural Language Processing Scientist/Engineer                     
4  NLP and Machine Learning Graduate                                         
5  AI Research - Natural Language Processing(NLP) Scientist                  
6  Analytical Linguist, Google AI                                            
7  IBM Research AI Residency Program                                         
8  AI Research Engineer – Natural Language Processing                        
9  Software Engineer: Machine Learning - Natural Language Processing (NLP)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

Much better, our job titles look really nice at this point. Onward!

In [19]:
X['job_description'] = X['job_description'].replace(r'<div class="jobsearch-JobComponent-description icl-u-xs-mt--md">', '', regex=True)


In [20]:
X.sample(10)

id                                                          title  \
1701  4588  Data Scientist/Senior Data Scientist                            
2812  6721  Sr Analytic Scientist (VIA)                                     
3220  7304  Data Science Manager                                            
1963  5601  Software Engineer Intern - Data (2019 Summer Internship)        
973   2283  Senior Data Scientist, Supply Chain                             
774   2010  Statistical Modeling Analyst                                    
1900  5513  Director of LABS                                                
790   2035  Applied Research Engineer                                       
2903  6871  Molder Utility A – CDL Driver (2 positions 2nd and 3rd Shift)   
1818  5211  System Engineer (Algorithm Focus)                               

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [21]:
len(X)

3701

In [22]:
#This will get rid of most of the html markers that are within <>
def clean_html(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [23]:
start = time.time()
for text in X:
    for i in range(len(X)):
        X['job_description'][i] = clean_html(X['job_description'][i])
end = time.time()
print("Done in", end - start, "seconds")

Done in 4.062126159667969 seconds


In [24]:
X.sample(25)

id  \
265   529    
588   1041   
1667  4536   
1970  5609   
3210  7293   
2093  5775   
575   1025   
1000  2655   
2638  6499   
199   397    
767   1998   
2802  6706   
916   2204   
3212  7295   
96    230    
1978  5618   
3485  7672   
774   2010   
2189  5887   
3258  7358   
1172  3037   
2305  6041   
3172  7249   
3087  7142   
3525  7726   

                                                                    title  \
265   IT Application Analyst                                                
588   Data Scientist                                                        
1667  Voice of Customer Manager                                             
1970  Applied Scientist                                                     
3210  Converged Infrastructure Reliability Engineer                         
2093  Senior Software Engineer, Labs                                        
575   Marketing Data Analyst                                                
1000  Lead Artifical Intelligence/Machine Learning Scientist PhD REQUIRED   
2638  Application Support Analyst                                           
199   Data Science Researcher                                               
767   Machine Learning Cloud Consultant                                     
2802  Associate - Tracy                                                     
916   Staff Algorithm Development Engineer                                  
3212  Engineer I                                                            
96    NLP Scientist/Engineer                                                
1978  Cyber Threat Analyst                                                  
3485  Research Intern, Silicon (PhD University Student)                     
774   Statistical Modeling Analyst                                          
2189  Senior Back End Java Developer                                        
3258  Computer Vision / Neural Network Specialist                           
1172  Machine Learning Engineer                                             
2305  Business Intelligence Architect for Digital Transformation            
3172  ML Specialist Solutions Architect                                     
3087  Quantitative Research Analyst                                         
3525  Machine Learning and Artificial Intelligence Scientist                

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

That worked like a charm! Let's continue cleaning.

In [25]:
X['job_description'] = X['job_description'].replace(r'\n', ' ', regex=True)

In [26]:
X.sample(25)

id  \
748   1973   
3296  7436   
2205  5909   
170   349    
3467  7650   
111   256    
2088  5770   
891   2171   
35    102    
3229  7313   
3033  7075   
2320  6059   
2836  6760   
1104  2892   
13    57     
3348  7503   
122   273    
621   1083   
1143  2981   
774   2010   
851   2123   
2344  6086   
3431  7611   
3472  7657   
1438  3428   

                                                                       title  \
748   Early Innovation Lead Embedded and Algorithm Engineer                    
3296  Data Science Specialist                                                  
2205  Senior Software Engineer - Technology                                    
170   Business Analyst - Entry-Level - Tech Startup                            
3467  RESEARCH FELLOW - Primary Cilia in Neural and Craniofacial Development   
111   Computational Linguist, NLU for Google Assistant                         
2088  Engagement Data Engineer                                                 
891   Assistant Buyer - Ladies Apparel                                         
35    Freelance Video Producer (Remote)                                        
3229  Senior SDE, Alexa Artificial Intelligence -- New Initiative              
3033  Lead Java Software Engineer                                              
2320  Sr. HPC Software Engineer (Clearance Required)                           
2836  Contact Center Business Analyst                                          
1104  Data Analytics Rotation and Training Program                             
13    Associate Data Scientist                                                 
3348  Data Science Internship                                                  
122   NLP/Python Developer                                                     
621   Data Scientist, Operations &amp; Manufacturing                           
1143  Research Analyst and QC                                                  
774   Statistical Modeling Analyst                                             
851   Data Scientist                                                           
2344  Software Development Engineer in Test (Mid to Senior)                    
3431  Embedded Software &amp; Hardware Engineer                                
3472  Summer 2019 Software Engineer Intern                                     
1438  Machine Learning Engineer-Defense                                        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [27]:
X['job_description'] = X['job_description'].replace(r'&amp;', ' ', regex=True)

In [28]:
X['title'] = X['title'].replace(r'&amp;', ' ', regex=True)

In [29]:
X.sample(25)

id  \
1402  3388   
219   449    
3036  7078   
3274  7392   
3389  7563   
183   369    
2705  6585   
3304  7447   
1086  2852   
2615  6461   
1678  4550   
3543  7746   
1782  5012   
2736  6621   
3678  7888   
3027  7052   
3028  7053   
725   1938   
3455  7636   
3489  7679   
2142  5836   
550   990    
798   2046   
2022  5683   
2778  6672   

                                                                           title  \
1402  Machine Learning Engineer                                                    
219   Cognitive Delivery Lead                                                      
3036  Lead Maintenance Supervisor                                                  
3274  Data Scientist                                                               
3389  Software Developer, Applications                                             
183   2019 Summer Internship- Technology                                           
2705  Senior Market Intelligence Analyst                                           
3304  Deep Learning Research Intern                                                
1086  Developmental Intern - Data Analytics                                        
2615  Dock Hand - Natick Boathouse                                                 
1678  AI Software Developer:Veteran+Military Connected                             
3543  Advanced Systems Lab - Neural Processor (Deep Learning) Research Scientist   
1782  Data Engineer II                                                             
2736  Medical Transcriptionist II (Clackamas)                                      
3678  Radar Machine Learning Scientist for Autonomous Driving                      
3027  Physician - Pulmonologist                                                    
3028  Senior Software Development Engineer, Alexa AI, Seattle                      
725   Data Scientist                                                               
3455  Research Scientist, Natural Language Processing                              
3489  Enterprise Data Scientist I                                                  
2142  Senior Data Engineer                                                         
550   Business Intelligence   Insights Analyst                                     
798   Analyst (Recruiting/Sourcing)                                                
2022  Java Backend Developer                                                       
2778  Software Development Engineer, Alexa Artificial Intelligence                 

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [30]:
X['job_description'] = X['job_description'].replace(r'>', ' ', regex=True)

In [31]:
X.sample(25)

id  \
1287  3217   
2816  6725   
1031  2720   
2876  6828   
1849  5401   
895   2176   
3515  7710   
2268  5998   
3278  7397   
2903  6871   
25    83     
1010  2679   
2843  6767   
3436  7617   
1245  3153   
1944  5574   
1998  5648   
3139  7207   
1043  2757   
2200  5904   
2988  6970   
3392  7566   
1767  4968   
1332  3285   
1703  4591   

                                                                                    title  \
1287  Machine Learning Manager, Alexa Experiences                                           
2816  Principal Engineering Manager                                                         
1031  Data Science Graduate Internship                                                      
2876  Senior Software Development Manager - Alexa AI                                        
1849  Big Data/Hadoop Developer                                                             
895   Senior Applied Scientist                                                              
3515  Machine Learning/Computational Neuroscience                                           
2268  Sr. Big Data Engineer                                                                 
3278  Product Marketing Specialist                                                          
2903  Molder Utility A – CDL Driver (2 positions 2nd and 3rd Shift)                         
25    NLP Scientist                                                                         
1010  Machine Learning intern                                                               
2843  Deep Learning Software Engineer - AI Applications                                     
3436  Data Scientist                                                                        
1245  Data Scientist, Applied Machine Learning                                              
1944  Software Engineer - Machine Learning / R                                              
1998  Data Engineer - Streaming                                                             
3139  Data Scientist                                                                        
1043  Intern, Experience Design for Machine Learning                                        
2200  Data Scientist - Decision Sciences Personal Valuation team                            
2988  Sr Manager - Infr Architect                                                           
3392  Postdoctoral Research Scientist, People AR (PhD Student or Grad)                      
1767  Machine Learning Engineer                                                             
1332  Data Scientist – Natural Language Processing and Machine Learning                     
1703  Data Science Director, Gartner Research and Advisory, Stamford, CT or Arlington, VA   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

The last regex left quite a bit of whitespace, which isn't useful. Let's strip that before moving on.

In [32]:
X['job_description'] = X['job_description'].str.strip()

In [33]:
X.sample(25)

id  \
3199  7280   
3484  7671   
323   632    
2900  6868   
292   575    
3648  7856   
1066  2806   
3037  7079   
3517  7713   
1328  3280   
692   1890   
2964  6942   
51    148    
923   2216   
3149  7218   
266   530    
1444  3438   
1862  5466   
1081  2837   
1191  3072   
3217  7301   
339   657    
2672  6547   
3415  7592   
714   1922   

                                                                               title  \
3199  Data Scientist                                                                   
3484  Specialist, Model Development                                                    
323   Computational Linguist Researcher                                                
2900  Contact Center Solutions Architect                                               
292   Business Program Intern                                                          
3648  AI Research Scientist                                                            
1066  Machine Learning Engineering                                                     
3037  Senior / Lead / Principal Data Scientist / Data Science Engineer (Experienced)   
3517  Sr. Machine Learning Engineer                                                    
1328  Applied Machine Learning Engineer                                                
692   Research Analyst                                                                 
2964  Senior Software Engineer                                                         
51    General - NLP Scientist                                                          
923   Systems Integration   Test Engineer, Orlando, FL                                 
3149  Autonomy AI Role - AI / ML Manager                                               
266   Research Scientist - NLP                                                         
1444  Data Scientist                                                                   
1862  Systems Engineer / DevOps                                                        
1081  Applied Machine Learning Scientist                                               
1191  Head Customer Data Platform   AI                                                 
3217  Senior Research SDE                                                              
339   Ontologist / Semantic Modeling Expert (HCE)                                      
2672  Sr. Applied Scientist                                                            
3415  Solutions Engineer                                                               
714   TECHNOLOGIST-NEURO II                                                            

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [34]:
#We don't need the ids that populated in the SQLite Database
X = X.drop(['id'], axis=1)

In [35]:
X.head(10)

title  \
0  Research Scientist - Natural Language Processing                          
1  Natural Language Processing Engineer, New York City                       
2  Research Programmer I - Natural Language Processing                       
3  Siri - Natural Language Processing Scientist/Engineer                     
4  NLP and Machine Learning Graduate                                         
5  AI Research - Natural Language Processing(NLP) Scientist                  
6  Analytical Linguist, Google AI                                            
7  IBM Research AI Residency Program                                         
8  AI Research Engineer – Natural Language Processing                        
9  Software Engineer: Machine Learning - Natural Language Processing (NLP)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [36]:
#Let's remove punctuation.
def punctuation_remover(row):
    punctuation = '!"#$&\'()*+,-./:;<=>?@[\\]^_`{|}~«»'  
    
    punctuationNoPeriod = "[" + re.sub("\.","", punctuation) + "]"
    row = re.sub(punctuationNoPeriod, " ", str(row))
    #to remove double white spaces and create space after %,
    row = row.replace('  ', ' ')
    row = row.replace('%', '% ')
    return row.strip()

<input>:5: DeprecationWarning:

invalid escape sequence \.

<input>:5: DeprecationWarning:

invalid escape sequence \.

<input>:5: DeprecationWarning:

invalid escape sequence \.

<ipython-input-36-b2830c59b60e>:5: DeprecationWarning:

invalid escape sequence \.



In [37]:
# remove punctuation from the job titles first
X['title'] = X['title'].apply(punctuation_remover)

In [38]:
X.head(10)

title  \
0  Research Scientist  Natural Language Processing                       
1  Natural Language Processing Engineer New York City                    
2  Research Programmer I  Natural Language Processing                    
3  Siri  Natural Language Processing Scientist Engineer                  
4  NLP and Machine Learning Graduate                                     
5  AI Research  Natural Language Processing NLP Scientist                
6  Analytical Linguist Google AI                                         
7  IBM Research AI Residency Program                                     
8  AI Research Engineer – Natural Language Processing                    
9  Software Engineer Machine Learning  Natural Language Processing NLP   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [39]:
#Since that worked well, let's now remove punctuation from the job descriptions.
X['job_description'] = X['job_description'].apply(punctuation_remover)

In [40]:
X.head(10)

title  \
0  Research Scientist  Natural Language Processing                       
1  Natural Language Processing Engineer New York City                    
2  Research Programmer I  Natural Language Processing                    
3  Siri  Natural Language Processing Scientist Engineer                  
4  NLP and Machine Learning Graduate                                     
5  AI Research  Natural Language Processing NLP Scientist                
6  Analytical Linguist Google AI                                         
7  IBM Research AI Residency Program                                     
8  AI Research Engineer – Natural Language Processing                    
9  Software Engineer Machine Learning  Natural Language Processing NLP   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [41]:
X['gender_breakdown'] = X['job_class']

In [42]:
X.job_class.unique()

array(['Natural Language Processing', 'Text Mining',
       'Pattern Recognition', 'Machine Learning', 'Text Analytics',
       'Apache Spark', 'Speech Recognition', 'Neural Networks'],
      dtype=object)

In [43]:
X.loc[X['gender_breakdown'] == 'Natural Language Processing', 'gender_breakdown'] = 1
X.loc[X['gender_breakdown'] == 'Text Mining', 'gender_breakdown'] = 1
X.loc[X['gender_breakdown'] == 'Text Analytics', 'gender_breakdown'] = 1
X.loc[X['gender_breakdown'] == 'Speech Recognition', 'gender_breakdown'] = 1
X.loc[X['gender_breakdown'] == 'Pattern Recognition', 'gender_breakdown'] = 0
X.loc[X['gender_breakdown'] == 'Machine Learning', 'gender_breakdown'] = 0
X.loc[X['gender_breakdown'] == 'Apache Spark', 'gender_breakdown'] = 0
X.loc[X['gender_breakdown'] == 'Neural Networks', 'gender_breakdown'] = 0

In [44]:
X.sample(5)

,title,job_description,job_class,gender_breakdown
81,Applied Scientist Alexa AI,Graduate degree MS or PhD in Electrical Engineering Computer Sciences or Mathematics with specialization in speech recognition natural language processing or machine learning · Familiar with programming languages such as C C Java Perl or Python Amazon is looking for a passionate talented and inventive Scientist with a strong machine learning background to help build industry leading Speech and Language technology Our mission is to push the envelope in Automatic Speech Recognition ASR Natural Language Understanding NLU and Audio Signal Processing in order to provide the best possible experience for our customers As a Scientist you will work with talented peers to develop novel algorithms and modeling techniques to advance the state of the art in spoken language understanding Your work will directly impact our customers in the form of products and services that make use of speech and language technology You will leverage Amazon’s heterogeneous data sources and large scale computing resources to accelerate advances in spoken language understanding We are hiring in all areas of spoken language understanding ASR NLU text to speech TTS and Dialog Management,Natural Language Processing,1
1155,Junior Data Scientist,Overview and Responsibilities The Junior Data Science Engineer Advanced Media Analytics will be responsible for supporting the build out of new alternate audience platforms across Viacom Media Networks In this role the individual will contribute to innovation in audience measurement and help discover how VMN content is being consumed across platforms Identify ingest and enrich a wide range of structured and unstructured data into datasets for analysis Learn to apply statistical and advanced techniques e g segmentation machine learning to develop both prototypes and scalable i e efficient automated data analyses Conduct basic ad hoc analysis building models forecasting viewing behavior Extensive code development debugging optimizing and productionizing Support the development of machine learning pipeline architecture Help build out frameworks for quickly rolling out new data analysis for standalone data driven products and services to support our network of media brands Be able to transform unstructured raw data in to formats suitable for modeling Be able to work in teams and collaborate with stakeholders to define requirements Basic Qualifications Bachelor’s or Master’s degree in a quantitative field Knowledge of Python and or R Knowledge of algorithms for data mining machine learning and or natural language processing Possess some understanding of statistical procedures used in advanced analytics Experience processing large amounts of structured and unstructured data using Spark Hive or other Big Data technologies Experience building scalable data models and performing complex relational database queries using SQL Oracle MySQL etc Additional Qualifications Attention to detail and demonstrated ability to detect and resolve data analytics quality issues Outstanding verbal and written communication skills Experienced user of data visualization tools e g Tableau matplotlib ggplot2 etc,Machine Learning,0
46,Data Scientist I,Functional Area SM Sales Marketing Estimated Travel Percentage % No Travel Relocation Provided No American General Life Insurance Company Your future team Data Scientists collaborate with AIG’s Life and Retirement businesses across multiple functions including product distribution marketing operations and technology teams By leveraging cutting edge techniques like natural language processing and other machine learning and predictive modeling tools they solve both classic and new challenges in an industry with a long history They are passionate about challenging the status quo and driving transformational change Your contribution at AIG As an influencer at AIG people come to you as a “go to” source for help and support because of 

In [45]:
# to save for later
df1 = X

## Unsupervised Approach

I'll use the Latend Dirichlet Allocation method first, and then use it with bigrams and trigrams. I will evalute for coherence, and tune the models.

In [46]:
#First step, tokenizer.
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [47]:
nltk.download('wordnet')
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tiffanyfrench/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [48]:
#nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
stop_words = en_stop.union(set(['machine','business','analytics','learning','technology',
                              'system', 'science', 'research', 'analysis', 'experience',
                               'customer', 'include', 'plant', 'clerkship', 'language',
                               'university', 'google', 'hadoop', 'spark', 'apache', 'microsoft',
                               'ai', 'ml', 'nlp', 'speech', 'natural', 'engineers', 'cloud',
                               'intelligence', 'aws','ml', 'nlp', 'ai', 'azure', 'amazon']))

In [49]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [50]:
import random
text_data = []
for text in X:
    for i in range(len(X)):
        tokens = prepare_text_for_lda(X['job_description'][i])
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['believe', 'great', 'writing', 'outcome', 'focus', 'alive', 'idea', 'going', 'start', 'finish', 'document', 'three', 'minutes', 'knowing', 'respond', 'anything', 'write', 'press', 'writing', 'software', 'idea', 'center', 'certain', 'potential', 'datasets', 'everywhere', 'uncover', 'hide', 'insight', 'trend', 'story', 'people', 'strong', 'storyteller', 'textio', 'story', 'world', 'success', 'engineer', 'textio', 'skill', 'important', 'anything', 'excel', 'telling', 'compel', 'story', 'immerse', 'valuable', 'asset', 'customer', 'result', 'success', 'engineering', 'persuade', 'convince', 'build', 'relationship', 'across', 'worldwide', 'closely', 'customers’', 'textio', 'predictive', 'engine', 'pattern', 'share', 'textio', 'insight', 'company', 'usage', 'competitive', 'positioning', 'partner', 'closely', 'product', 'team', 'ensure', 'textio', 'reflect', 'happening', 'today', 'linguistic', 'landscape', 'level', 'customer', 'include', 'senior', 'executive', 'fortune', 'company', 'phenomenal

['kasisto', 'enable', 'financial', 'institution', 'service', 'engage', 'acquire', 'customer', 'human', 'intelligent', 'conversation', 'anytime', 'anywhere', 'kasisto', 'conversational', 'platform', 'power', 'channel', 'virtual', 'assistant', 'chatbots', 'fluent', 'banking', 'across', 'mobile', 'website', 'messaging', 'platform', 'voice', 'enable', 'devices', 'leading', 'conversational', 'platform', 'finance', 'industry', 'kasisto', 'customer', 'wells', 'fargo', 'morgan', 'mastercard', 'standard', 'charter', 'among', 'others', 'choose', 'banking', 'prove', 'track', 'record', 'drive', 'result', 'improve', 'experience', 'platform', 'engage', 'million', 'consumer', 'around', 'world', 'across', 'multiple', 'channels', 'different', 'language', 'optimize', 'performance', 'scalability', 'security', 'compliance', 'build', 'deep', 'portfolio', 'industry', 'include', 'platform', 'tool', 'services', 'customize', 'continually', 'improve', 'consumer', 'experience', 'seamlessly', 'feature', 'channels

['early', 'stage', 'start', 'found', 'marissa', 'mayer', 'enrique', 'muñoz', 'torres', 'base', 'downtown', 'focus', 'building', 'consumer', 'application', 'enable', 'artificial', 'looking', 'software', 'strong', 'background', 'artificial', 'thrive', 'explore', 'datasets', 'ambiguous', 'problem', 'space', 'early', 'member', 'comfortable', 'working', 'pace', 'environment', 'enjoy', 'apply', 'domain', 'fresh', 'idea', 'finding', 'novel', 'potentially', 'cutting', 'artificial', 'everyday', 'problem', 'sound', 'interest', 'please', 'consider', 'apply', 'requirement', 'graduate', 'degree', 'computer', 'artificial', 'relate', 'technical', 'field', 'years', 'relevant', 'software', 'development', 'following', 'processing', 'image', 'processing', 'pattern', 'recognition', 'classification', 'recommendation', 'system', 'ranking', 'system', 'similar', 'starting', 'scientific', 'thinker', 'love', 'experiment', 'datasets', 'enjoy', 'invent', 'solution', 'ambiguity', 'defining', 'product', 'specs', 'i

['masters', 'degree', 'computer', 'engineering', 'mathematics', 'relate', 'discipline', 'least', 'years', 'system', 'least', 'years', 'building', 'base', 'production', 'software', 'hands', 'development', 'python', 'equal', 'opportunity', 'affirmative', 'action', 'employer', 'minority', 'female', 'disability', 'veteran', 'gender', 'identity', 'sexual', 'orientation', 'search', 'create', 'powerful', 'focus', 'search', 'advertising', 'solution', 'technology', 'whenever', 'visit', 'worldwide', 'type', 'query', 'browse', 'product', 'category', 'search', 'services', 'design', 'develop', 'deploy', 'performance', 'fault', 'tolerant', 'distribute', 'search', 'system', 'million', 'customer', 'every', 'search', 'relevance', 'works', 'maximize', 'quality', 'effectiveness', 'search', 'visitor', 'website', 'worldwide', 'product', 'search', 'organization', 'mission', 'customer', 'efficiently', 'anything', 'looking', 'charter', 'require', 'build', 'solution', 'beyond', 'lexical', 'match', 'semanticall

['verizon', 'innovate', 'constantly', 'big', 'names', 'quick', 'smart', 'take', 'vision', 'take', 'focus', 'apply', 'every', 'bright', 'mind', 'latest', 'technology', 'pioneer', 'process', 'combining', 'enhance', 'critical', 'function', 'within', 'digital', 'finance', 'center', 'excellence', 'finding', 'value', 'provide', 'strategic', 'support', 'specifically', 'leveraging', 'disruptive', 'finance', 'analystwill', 'support', 'integration', 'within', 'support', 'project', 'management', 'developing', 'practice', 'drive', 'consistency', 'adoption', 'emerge', 'tool', 'processing', 'cognitive', 'technology', 'list', 'additional', 'responsibility', 'coordinate', 'aspect', 'project', 'lifecycle', 'relate', 'implementation', 'disruptive', 'tool', 'processing', 'others', 'assist', 'large', 'scale', 'complex', 'profile', 'project', 'duration', 'budget', 'million', 'deliver', 'program', 'enhancement', 'deliverable', 'significant', 'impact', 'verizon', 'finance', 'landscape', 'flexible', 'collabor

...

['master', 'degree', 'computer', 'equivalent', 'years', 'bachelor', 'degree', 'computer', 'equivalent', 'years', 'software', 'development', 'include', 'design', 'implementation', 'debug', 'support', 'excellent', 'computer', 'fundamentals', 'algorithm', 'design', 'structure', 'problem', 'solving', 'complexity', 'proficiency', 'developing', 'object', 'orient', 'software', 'relevant', 'language', 'enjoy', 'working', 'large', 'scale', 'distribute', 'system', 'influence', 'world', 'interact', 'analyze', 'large', 'across', 'number', 'industry', 'case', 'provide', 'manage', 'framework', 'make', 'effective', 'process', 'amount', 'across', 'dynamically', 'scalable', 'instance', 'using', 'popular', 'distribute', 'framework', 'hbase', 'presto', 'flink', 'interact', 'store', 'dynamodb', 'customer', 'variety', 'industry', 'estate', 'travel', 'finance', 'gaming', 'process', 'interact', 'datasets', 'fulfill', 'exist', 'uncover', 'need', 'looking', 'talented', 'develop', 'innovative', 'feature', 'enab

['overview', 'seeking', 'senior', 'engineer', 'client', 'location', 'washington', 'union', 'station', 'responsibility', 'closely', 'solution', 'architect', 'provide', 'overall', 'engineering', 'support', 'establish', 'environment', 'include', 'limited', 'build', 'deliver', 'integrate', 'government', 'exist', 'catalog', 'prototype', 'migrate', 'build', 'application', 'premise', 'computing', 'platform', 'leveraging', 'technology', 'engineer', 'closely', 'user', 'collect', 'build', 'cleanse', 'assemble', 'refine', 'datasets', 'support', 'variety', 'need', 'forward', 'stakeholder', 'include', 'scientist', 'economist', 'enforcement', 'attoneys', 'examiner', 'analyst', 'engineer', 'closely', 'administrator', 'network', 'database', 'engineering', 'security', 'engineering', 'team', 'comply', 'security', 'policy', 'procedure', 'trouble', 'shoot', 'resolve', 'issue', 'engineering', 'delivery', 'support', 'technology', 'devops', 'qualification', 'require', 'skill', 'qualification', 'computer', 'm

['summary', 'post', 'number', '200018211', 'revolution', 'human', 'computer', 'interaction', 'supply', 'product', 'redefine', 'mobile', 'computing', 'create', 'groundbreaking', 'large', 'scale', 'system', 'speak', 'artificial', 'people', 'create', 'intelligent', 'assistant', 'help', 'million', 'people', 'things', 'asking', 'apple', 'looking', 'exceptionally', 'skilled', 'creative', 'scientist', 'eager', 'involve', 'hands', 'improve', 'apply', 'qualification', 'strong', 'background', 'dialogue', 'processing', 'field', 'depth', 'hands', 'strong', 'software', 'engineering', 'ability', 'ideally', 'python', 'passion', 'building', 'prototype', 'turning', 'production', 'quality', 'design', 'implementation', 'relate', 'toolkits', 'tensorflow', 'pytorch', 'speaker', 'recognition', 'relate', 'technology', 'curious', 'technology', 'passionate', 'explore', 'case', 'description', 'looking', 'exceptional', 'individual', 'extend', 'understand', 'learn', 'remember', 'cross', 'functional', 'consist', '

['great', 'people', 'great', 'benefit', 'guest', 'services', 'hire', 'dining', 'supervisor', 'oversee', 'front', 'house', 'dining', 'operations', 'plan', 'senior', 'living', 'community', 'kansas', 'qualify', 'candidate', 'leadership', 'service', 'industry', 'service', 'orient', 'friendly', 'motivating', 'hospitality', 'senior', 'living', 'prefer', 'apply', 'dining', 'manager', 'summary', 'dining', 'supervisor', 'assist', 'general', 'manager', 'aspect', 'floor', 'management', 'include', 'beverage', 'equipment', 'property', 'facilitate', 'fulfillment', 'financial', 'goal', 'company', 'initiative', 'addition', 'dining', 'supervisor', 'assist', 'cultivate', 'environment', 'provide', 'exceptional', 'service', 'ensure', 'staff', 'member', 'perform', 'consistently', 'level', 'dining', 'supervisor', 'works', 'towards', 'motivating', 'employee', 'instilling', 'accountability', 'achieve', 'result', 'drive', 'success', 'position', 'essential', 'function', 'oversee', 'shift', 'ensure', 'guest', 's

['description', 'responsibility', 'growing', 'energy', 'starting', 'engineer', 'hands', 'artificial', 'interactive', 'broker', 'place', 'pride', 'provide', 'global', 'client', 'cutting', 'trading', 'application', 'interactive', 'broker', 'found', 'software', 'engineer', 'rewrite', 'rule', 'trading', 'automate', 'anything', 'everything', 'software', 'development', 'lifeblood', 'show', 'latest', 'command', 'respect', 'competitive', 'compensation', 'package', 'develop', 'solution', 'world', 'large', 'scale', 'problem', 'using', 'artificial', 'improve', 'engine', 'services', 'brokerage', 'client', 'http', 'bridges', 'software', 'engineer', 'scientist', 'skill', 'neural', 'network', 'define', 'training', 'validation', 'automate', 'test', 'setup', 'feedback', 'behavior', 'base', 'interaction', 'formulate', 'experiment', 'improve', 'comprehension', 'altering', 'topology', 'tweak', 'activation', 'function', 'alike', 'assist', 'coding', 'execution', 'engine', 'qualification', 'degree', 'compute

In [51]:
# Using Gensim to bolster NLTK

In [52]:
dictionary = corpora.Dictionary(text_data)

2019-03-03 16:32:14,541 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-03-03 16:32:14,590 : INFO : built Dictionary(4348 unique tokens: ['abstract', 'accompany', 'account', 'across', 'action']...) from 139 documents (total 34789 corpus positions)


In [53]:
dictionary.filter_extremes(no_below=10, no_above=0.2)

2019-03-03 16:32:14,606 : INFO : discarding 3881 tokens: [('abstract', 5), ('accompany', 2), ('across', 53), ('alive', 2), ('always', 6), ('anything', 3), ('asset', 5), ('background', 30), ('benefit', 50), ('build', 62)]...
2019-03-03 16:32:14,608 : INFO : keeping 467 tokens which were in no less than 10 and no more than 27 (=20.0%) documents
2019-03-03 16:32:14,611 : INFO : resulting dictionary: Dictionary(467 unique tokens: ['account', 'action', 'audience', 'become', 'believe']...)


In [54]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [55]:
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

2019-03-03 16:32:14,702 : INFO : saving Dictionary object under dictionary.gensim, separately None
2019-03-03 16:32:14,749 : INFO : saved dictionary.gensim


In [56]:
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
# Convert to array
docs = array(X['job_description'])

In [57]:
# Define function for tokenize and lemmatizing
def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [58]:
# Perform function on our document
docs = docs_preprocessor(docs)

In [59]:
# Create Biagram & Trigram Models 
# Add bigrams and trigrams to docs,minimum count 10 means only that appear 10 times or more.
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

2019-03-03 16:32:22,095 : INFO : collecting all words and their counts
2019-03-03 16:32:22,099 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-03-03 16:32:24,579 : INFO : collected 422880 word types from a corpus of 1335495 words (unigram + bigrams) and 3701 sentences
2019-03-03 16:32:24,579 : INFO : using 422880 counts as vocab in Phrases<0 vocab, min_count=10, threshold=10.0, max_vocab_size=40000000>
2019-03-03 16:32:24,580 : INFO : collecting all words and their counts
2019-03-03 16:32:24,585 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-03-03 16:32:32,377 : INFO : collected 472563 word types from a corpus of 1148603 words (unigram + bigrams) and 3701 sentences
2019-03-03 16:32:32,378 : INFO : using 472563 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


In [60]:
# Remove rare & common tokens 
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
dictionary.filter_extremes(no_below=10, no_above=0.2)

2019-03-03 16:32:45,197 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-03-03 16:32:47,312 : INFO : built Dictionary(33669 unique tokens: ['able', 'about', 'abreast', 'access', 'accommodation']...) from 3701 documents (total 1921338 corpus positions)
2019-03-03 16:32:47,420 : INFO : discarding 22851 tokens: [('able', 881), ('about', 1591), ('algorithm', 1410), ('analysis', 1719), ('analytics', 1395), ('applicant', 1216), ('application', 1817), ('apply', 1093), ('approach', 837), ('area', 1317)]...
2019-03-03 16:32:47,421 : INFO : keeping 10818 tokens which were in no less than 10 and no more than 740 (=20.0%) documents
2019-03-03 16:32:47,446 : INFO : resulting dictionary: Dictionary(10818 unique tokens: ['abreast', 'access', 'accommodation', 'accommodation_apply', 'accommodation_request']...)


In [61]:
#Create dictionary and corpus required for Topic Modeling
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))
print(corpus[:1])

Number of unique tokens: 10818
Number of documents: 3701
[[(0, 1), (1, 1), (2, 3), (3, 1), (4, 3), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 6), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 3), (25, 1), (26, 3), (27, 1), (28, 1), (29, 2), (30, 3), (31, 1), (32, 1), (33, 1), (34, 3), (35, 1), (36, 1), (37, 3), (38, 2), (39, 1), (40, 1), (41, 4), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 3), (53, 1), (54, 1), (55, 3), (56, 1), (57, 1), (58, 1), (59, 2), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 3), (69, 1), (70, 2), (71, 3), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 2), (89, 4), (90, 1), (91, 2), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 2), (102, 1), (103, 1), (104, 1), (

In [62]:
# Set parameters.
num_topics = 5
chunksize = 500 
passes = 20 
iterations = 400
eval_every = 1  

In [63]:
# Make a index to word dictionary.
temp = dictionary[0]  # only to "load" the dictionary.
id2word = dictionary.id2token

lda_model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

2019-03-03 16:32:48,968 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
2019-03-03 16:32:48,971 : INFO : using serial LDA version on this node
2019-03-03 16:32:48,981 : INFO : running online (multi-pass) LDA training, 5 topics, 20 passes over the supplied corpus of 3701 documents, updating model once every 500 documents, evaluating perplexity every 500 documents, iterating 400x with a convergence threshold of 0.001000
2019-03-03 16:32:52,667 : INFO : -9.821 per-word bound, 904.3 perplexity estimate based on a held-out corpus of 500 documents with 125353 words
2019-03-03 16:32:52,668 : INFO : PROGRESS: pass 0, at document #500/3701
2019-03-03 16:32:55,517 : INFO : optimized alpha [0.053372353, 0.12568635, 0.1441711, 0.031060174, 0.086538546]
2019-03-03 16:32:55,519 : INFO : merging changes from 500 documents into a model of 3701 documents
2019-03-03 16:32:55,525 : INFO : topic #0 (0.053): 0.004*"collaborate_with" + 0.004*"data_scientist" + 0.003*"team_member" + 0

2019-03-03 16:33:02,329 : INFO : topic diff=1.180101, rho=0.500000
2019-03-03 16:33:03,440 : INFO : -10.072 per-word bound, 1076.2 perplexity estimate based on a held-out corpus of 500 documents with 154558 words
2019-03-03 16:33:03,441 : INFO : PROGRESS: pass 0, at document #2500/3701
2019-03-03 16:33:04,119 : INFO : optimized alpha [0.1343933, 0.25697705, 0.35539693, 0.076541886, 0.1331344]
2019-03-03 16:33:04,120 : INFO : merging changes from 500 documents into a model of 3701 documents
2019-03-03 16:33:04,126 : INFO : topic #0 (0.134): 0.004*"best_practice" + 0.003*"enterprise" + 0.003*"advanced_analytics" + 0.003*"data_warehouse" + 0.003*"team_member" + 0.003*"collaborate_with" + 0.003*"sale" + 0.002*"apache" + 0.002*"cross_functional" + 0.002*"text_analytics"
2019-03-03 16:33:04,127 : INFO : topic #1 (0.257): 0.006*"least_year" + 0.004*"data_scientist" + 0.003*"bachelor_degree" + 0.003*"each_other" + 0.003*"collaborate_with" + 0.003*"data_mining" + 0.002*"preferred_qualification"

2019-03-03 16:33:09,952 : INFO : topic #1 (0.328): 0.006*"data_scientist" + 0.005*"computer_vision" + 0.004*"least_year" + 0.004*"artificial_intelligence" + 0.004*"neural_network" + 0.003*"data_mining" + 0.003*"collaborate_with" + 0.003*"preferred_qualification" + 0.003*"real_time" + 0.002*"speech_recognition"
2019-03-03 16:33:09,953 : INFO : topic #2 (0.457): 0.006*"data_scientist" + 0.005*"software_development" + 0.005*"neural_network" + 0.005*"large_scale" + 0.004*"software_engineer" + 0.003*"this_role" + 0.003*"familiarity_with" + 0.003*"cutting_edge" + 0.003*"artificial_intelligence" + 0.003*"team_member"
2019-03-03 16:33:09,955 : INFO : topic #3 (0.074): 0.003*"data_set" + 0.003*"others" + 0.003*"rule" + 0.003*"must_able" + 0.002*"guest" + 0.002*"problem_solving" + 0.002*"verbal_communication" + 0.002*"essential_function" + 0.002*"writing" + 0.002*"active_listening"
2019-03-03 16:33:09,957 : INFO : topic #4 (0.162): 0.005*"neural_network" + 0.004*"student" + 0.004*"artificial_int

2019-03-03 16:33:15,576 : INFO : topic #4 (0.170): 0.004*"internship" + 0.004*"full_time" + 0.004*"university" + 0.004*"microsoft" + 0.004*"student" + 0.003*"around_world" + 0.003*"artificial_intelligence" + 0.003*"intern" + 0.003*"will_receive" + 0.003*"criminal_history"
2019-03-03 16:33:15,578 : INFO : topic diff=0.306027, rho=0.326129
2019-03-03 16:33:16,574 : INFO : -8.350 per-word bound, 326.2 perplexity estimate based on a held-out corpus of 500 documents with 214331 words
2019-03-03 16:33:16,575 : INFO : PROGRESS: pass 1, at document #3000/3701
2019-03-03 16:33:17,009 : INFO : optimized alpha [0.22548084, 0.32737806, 0.4222062, 0.08226404, 0.15849876]
2019-03-03 16:33:17,011 : INFO : merging changes from 500 documents into a model of 3701 documents
2019-03-03 16:33:17,017 : INFO : topic #0 (0.225): 0.005*"speech_recognition" + 0.004*"policy_procedure" + 0.004*"report" + 0.004*"care" + 0.003*"procedure" + 0.003*"essential_function" + 0.003*"information_idea" + 0.003*"sale" + 0.00

2019-03-03 16:33:22,039 : INFO : merging changes from 500 documents into a model of 3701 documents
2019-03-03 16:33:22,046 : INFO : topic #0 (0.211): 0.005*"health" + 0.004*"report" + 0.004*"sale" + 0.004*"care" + 0.004*"clinical" + 0.004*"policy" + 0.004*"policy_procedure" + 0.004*"procedure" + 0.003*"patient" + 0.003*"duty"
2019-03-03 16:33:22,047 : INFO : topic #1 (0.422): 0.008*"data_scientist" + 0.006*"data_mining" + 0.005*"computer_vision" + 0.004*"mining" + 0.004*"neural_network" + 0.004*"data_set" + 0.004*"least_year" + 0.004*"text_mining" + 0.004*"predictive" + 0.004*"artificial_intelligence"
2019-03-03 16:33:22,049 : INFO : topic #2 (0.557): 0.006*"data_scientist" + 0.005*"software_development" + 0.004*"large_scale" + 0.004*"software_engineer" + 0.003*"neural_network" + 0.003*"this_role" + 0.003*"familiarity_with" + 0.003*"cutting_edge" + 0.003*"team_member" + 0.003*"data_set"
2019-03-03 16:33:22,052 : INFO : topic #3 (0.070): 0.004*"others" + 0.003*"essential_function" + 0.0

...

2019-03-03 16:35:33,473 : INFO : topic #3 (0.070): 0.006*"others" + 0.004*"essential_function" + 0.004*"equipment" + 0.004*"rule" + 0.003*"high_school" + 0.003*"food" + 0.003*"guest" + 0.003*"comprehension" + 0.003*"understand_speech" + 0.003*"related_document"
2019-03-03 16:35:33,475 : INFO : topic #4 (0.227): 0.007*"computer_vision" + 0.005*"without_regard" + 0.005*"speech_recognition" + 0.005*"artificial_intelligence" + 0.004*"gender_identity" + 0.004*"internship" + 0.004*"qualified_applicant" + 0.004*"race_color" + 0.004*"other_characteristic" + 0.004*"signal_processing"
2019-03-03 16:35:33,476 : INFO : topic diff=0.250618, rho=0.198411
2019-03-03 16:35:34,157 : INFO : -8.269 per-word bound, 308.5 perplexity estimate based on a held-out corpus of 500 documents with 144175 words
2019-03-03 16:35:34,158 : INFO : PROGRESS: pass 17, at document #3500/3701
2019-03-03 16:35:34,431 : INFO : optimized alpha [0.26358858, 0.42333904, 0.5552452, 0.06938426, 0.24342728]
2019-03-03 16:35:34,432

2019-03-03 16:35:37,517 : INFO : PROGRESS: pass 18, at document #1500/3701
2019-03-03 16:35:37,793 : INFO : optimized alpha [0.24897519, 0.5099325, 0.63174367, 0.06534335, 0.28264773]
2019-03-03 16:35:37,794 : INFO : merging changes from 500 documents into a model of 3701 documents
2019-03-03 16:35:37,801 : INFO : topic #0 (0.249): 0.006*"health" + 0.005*"clinical" + 0.005*"care" + 0.004*"report" + 0.004*"patient" + 0.004*"policy" + 0.004*"duty" + 0.004*"medical" + 0.004*"sale" + 0.004*"procedure"
2019-03-03 16:35:37,803 : INFO : topic #1 (0.510): 0.014*"data_scientist" + 0.008*"data_mining" + 0.008*"data_set" + 0.006*"neural_network" + 0.005*"mining" + 0.005*"least_year" + 0.004*"advanced_analytics" + 0.004*"predictive" + 0.004*"master_degree" + 0.004*"statistical_analysis"
2019-03-03 16:35:37,805 : INFO : topic #2 (0.632): 0.006*"software_development" + 0.004*"large_scale" + 0.004*"software_engineer" + 0.004*"this_role" + 0.003*"real_time" + 0.003*"fast_paced" + 0.003*"cutting_edge" 

2019-03-03 16:35:42,401 : INFO : topic #2 (0.559): 0.007*"software_development" + 0.005*"apache_spark" + 0.005*"software_engineer" + 0.004*"large_scale" + 0.004*"apache" + 0.004*"this_role" + 0.003*"real_time" + 0.003*"open_source" + 0.003*"best_practice" + 0.003*"familiarity_with"
2019-03-03 16:35:42,403 : INFO : topic #3 (0.069): 0.005*"others" + 0.004*"equipment" + 0.004*"essential_function" + 0.004*"rule" + 0.004*"high_school" + 0.003*"guest" + 0.003*"food" + 0.003*"comprehension" + 0.003*"related_document" + 0.003*"understand_speech"
2019-03-03 16:35:42,405 : INFO : topic #4 (0.245): 0.009*"computer_vision" + 0.006*"neural_network" + 0.005*"artificial_intelligence" + 0.005*"without_regard" + 0.004*"gender_identity" + 0.004*"race_color" + 0.004*"internship" + 0.004*"other_characteristic" + 0.004*"speech_recognition" + 0.004*"qualified_applicant"
2019-03-03 16:35:42,406 : INFO : topic diff=0.103505, rho=0.194617
2019-03-03 16:35:42,686 : INFO : -8.217 per-word bound, 297.5 perplexit

2019-03-03 16:35:46,074 : INFO : topic diff=0.083657, rho=0.191033
2019-03-03 16:35:46,777 : INFO : -8.314 per-word bound, 318.2 perplexity estimate based on a held-out corpus of 500 documents with 141747 words
2019-03-03 16:35:46,778 : INFO : PROGRESS: pass 19, at document #2000/3701
2019-03-03 16:35:47,112 : INFO : optimized alpha [0.25956035, 0.5252829, 0.686262, 0.06475411, 0.27520743]
2019-03-03 16:35:47,114 : INFO : merging changes from 500 documents into a model of 3701 documents
2019-03-03 16:35:47,122 : INFO : topic #0 (0.260): 0.006*"health" + 0.004*"care" + 0.004*"clinical" + 0.004*"report" + 0.004*"policy" + 0.004*"patient" + 0.004*"medical" + 0.004*"sale" + 0.003*"duty" + 0.003*"risk"
2019-03-03 16:35:47,124 : INFO : topic #1 (0.525): 0.014*"data_scientist" + 0.008*"data_set" + 0.008*"data_mining" + 0.005*"advanced_analytics" + 0.005*"neural_network" + 0.005*"least_year" + 0.005*"mining" + 0.004*"predictive" + 0.004*"master_degree" + 0.004*"problem_solving"
2019-03-03 16:3

2019-03-03 16:35:51,749 : INFO : topic #2 (0.572): 0.007*"software_development" + 0.005*"apache_spark" + 0.004*"software_engineer" + 0.004*"large_scale" + 0.004*"this_role" + 0.004*"apache" + 0.003*"real_time" + 0.003*"familiarity_with" + 0.003*"open_source" + 0.003*"best_practice"
2019-03-03 16:35:51,751 : INFO : topic #3 (0.068): 0.005*"others" + 0.004*"equipment" + 0.004*"essential_function" + 0.004*"rule" + 0.003*"high_school" + 0.003*"guest" + 0.003*"food" + 0.003*"comprehension" + 0.003*"related_document" + 0.003*"understand_speech"
2019-03-03 16:35:51,760 : INFO : topic #4 (0.263): 0.010*"computer_vision" + 0.008*"neural_network" + 0.006*"artificial_intelligence" + 0.005*"without_regard" + 0.004*"gender_identity" + 0.004*"other_characteristic" + 0.004*"race_color" + 0.004*"internship" + 0.004*"qualified_applicant" + 0.004*"image"
2019-03-03 16:35:51,761 : INFO : topic diff=0.110004, rho=0.191033


In [64]:
# Print the Keyword in the 5 topics
#print(lda_model.print_topics())

In [77]:
id2word = gensim.corpora.Dictionary(text_data)

2019-03-03 16:50:11,450 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-03-03 16:50:11,493 : INFO : built Dictionary(4348 unique tokens: ['abstract', 'accompany', 'account', 'across', 'action']...) from 139 documents (total 34789 corpus positions)


In [74]:
#id2word.filter_extremes(no_below=20, no_above=.1)

2019-03-03 16:49:42,600 : INFO : discarding 4348 tokens: [('abstract', 5), ('accompany', 2), ('account', 13), ('across', 53), ('action', 27), ('alive', 2), ('always', 6), ('anything', 3), ('asset', 5), ('audience', 15)]...
2019-03-03 16:49:42,602 : INFO : keeping 0 tokens which were in no less than 20 and no more than 13 (=10.0%) documents
2019-03-03 16:49:42,605 : INFO : resulting dictionary: Dictionary(0 unique tokens: [])


In [78]:
bow_collection = list(map(id2word.doc2bow, text_data))

In [79]:
print(bow_collection)

[[(0, 1), (1, 1), (2, 1), (3, 3), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 3), (10, 2), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 4), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 2), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 2), (88, 1), (89, 1), (90, 2), (91, 1), (92, 2), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 3), (108, 1), (109, 1), (110, 1)

In [80]:
lda = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=100,
)

2019-03-03 16:50:18,795 : INFO : using symmetric alpha at 0.01
2019-03-03 16:50:18,798 : INFO : using symmetric eta at 0.01
2019-03-03 16:50:18,802 : INFO : using serial LDA version on this node
2019-03-03 16:50:18,928 : INFO : running online (single-pass) LDA training, 100 topics, 1 passes over the supplied corpus of 3701 documents, updating model once every 2000 documents, evaluating perplexity every 3701 documents, iterating 50x with a convergence threshold of 0.001000
2019-03-03 16:50:18,930 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2019-03-03 16:50:18,934 : INFO : PROGRESS: pass 0, at document #2000/3701
2019-03-03 16:50:23,292 : INFO : merging changes from 2000 documents into a model of 3701 documents
2019-03-03 16:50:23,428 : INFO : topic #62 (0.010): 0.005*"software_development" + 0.004*"this_role" + 0.004*"artificial_intelligence" + 0.003*"amazon" + 0.003*"music" + 0.003*"sale" + 0.003*"

In [81]:
one = '''Master’s degree in Computer Science, Statistics, Engineering, Mathematics or related field, or equivalent experience.
Education is likely to include one or more of the following: Doctorate/Maters/Bachelors.
Experiencing with programming in R, MATLAB, Python or similar scripting languages.
Excellent data visualization and written communication skills to translate complex models and analysis results into layman terms.
Experience in building NLP models.
Audible, an Amazon company, is the leading provider of premium digital spoken audio information and entertainment on the internet, offering customers a new way to enhance and enrich their lives every day. 
Audible Data Scientists are members of an interdisciplinary research team with an integral role in the design and integration of models to automate decision making throughout the business in every country. We empower the cutting-edge machine learning and deep learning techniques in the many areas of the business, including but not limited to Customer: segmentation, acquisition, retention, engagement; Product: customer experience optimization, simulation, testing and evaluation, Content: recommender system, natural language processing in text and voice, and International business etc. We translate business goals into agile, insightful analytics and seek to create value for both stakeholders and customers and deliver findings in a clear and actionable way. 
We are currently looking for a Data Scientist to support the ongoing development of Product vertical. The candidate will be expected to work closely with our team members on the design, development, deployment of ML/NLP/DS models which built upon the cutting-edge technologies. Additionally, we are seeking candidates with strong rigor in data sciences, engineering, creativity, curiosity, and great judgment. 

KEY RESPONSIBILITIES 
Identifying necessary, relevant, and novel data sources and acquiring data, which often means building the necessary SQL / ETL queries, import processes through various company specific interfaces for accessing Red Shift storage systems, but also by building relationships with stakeholders and counterparts all over the world in order to form trusting, functional relationships that provide for a sustainable flow and sharing of information.
Exploring data will occupy the largest portion of attention, and should be second nature in order to deeply understand the phenomenon being modeled, and the validity and reliability of the inputs, including but not limited to inspecting univariate distributions, exploring bivariate relationships, constructing appropriate transformations, and tracking down the source and meaning of anomalies when and where they arise.
Model building should draw from any approach that enhances accuracy and understanding including statistical modeling, mathematical modeling, network modeling, social network modeling, natural language processing, machine learning, algorithms, genetic algorithms, and neural networks.
Validating models against alternative approaches, expected and observed outcome, and numerous directly and indirectly relevant business defined key performance indicators.
Reviewing models of peers for the purpose of reducing and managing risk to the business, and maximizing improvement of business practice and customer experience.
Implementing models from the initial evaluation of the computational demands, accuracy, and reliability of the relevant ETL processes, and the integrity of the data sources in production, to the computational demands, accuracy, and reliability of the simpler scoring processes, to the computational demands, accuracy, and reliability of model training in higher and higher frequencies in the production environment.
Operationalization will include identifying the stakeholders in the life of a model score throughout the company and around the world, forming trusting bonds, understanding the phenomenon, optimizing the model output for integration to the practice, assuring “buy in” from practitioners as well as leaders, training for proper utilization and communication, and assessing proper utilization.
Model management will include developing sustainable, consumable, accurate, and impactful reporting on model inputs, model outputs, observed outputs, business impact, and key performance indicators.
Data scientists must be able to discuss their research in any level of detail with their peers, and with appropriate calibration to stakeholders in small and large group settings. They are expected to acquire support and partnership from personalities through the company, including engineering and research teams at Amazon globally. Successful data scientists are expected to influence and mentor each other, influence and mentor their peers globally, and influence and mentor their partners in the business in every country.'''

In [82]:
?lda.get_document_topics

In [83]:
bow_lda = id2word.doc2bow(gensim.utils.simple_preprocess(one))
topics = lda.get_document_topics(bow_lda, minimum_probability=0)

In [84]:
for key, contribution in topics:
    print(f'\n:: {key} ({contribution*100:.2}%)')
    for i, __ in lda.get_topic_terms(key, topn=5):
        print(f' {id2word[i]}', end=',')


:: 0 (0.0037%)
 recovery, super, otherwise,

KeyError: 7564

In [ ]:
??lda.get_document_topics

In [ ]:
??lda.inference

In [ ]:
bow_lda = id2word.doc2bow(gensim.utils.simple_preprocess(one))
gamma, phi = lda.inference([bow])
one_topics = gamma[0] / gamma[0].sum()
one_topics

In [ ]:
two = """Lab Summary:

Samsung is working towards a vision of providing world-class AI services on its industry-leading hardware platforms like the Galaxy phones, and home appliances like the smart refrigerators and TVs, smart washers and dryers, etc. Real-time AI services on these connected devices will enable users to interact with devices more naturally via spoken dialog, visual interaction, continuous learning, and big data analysis.

Position Summary:

We are looking for candidates interested in extending our capabilities on cross-lingual NLP and enabling novel NLP technologies. The ideal candidate will have strong software engineering skills. They should also have background in NLP systems, experience of deep learning, and experience working with massive amounts of data.

Responsibilities Include:

Develop prototypes and real-world applications and improve the performance with various technologies Integrate the resulting system/software into existing platform with high quality implementation

Experience Requirements:

Master or Ph.D. in Computer Science or related field
R&D experience in natural language processing, machine learning and related fields
R&D experience in information extraction and text mining technologies
Hands-on experience of deep learning technologies and familiar with state-of-the-art deep learning toolkits (e.g., TensorFlow, PyTorch, Keras, etc)
Experience in programming languages (C++, Python, Java, etc)
Strong problem solving skills
Good communication and team-working skills
Experience Preferred:

Experience in natural language processing, information extraction, text mining technologies and related fields
Experience in a front-end language like Elm, JavaScript, Objective-C or Swift
Experience in at least one front-end web development framework such as React, Angular, Rails, Django, Vue.js
Experience in HTTP APIs and RESTful web services Understanding of web browser APIs and behavior
Samsung is an EEO/Veterans/Disabled/LGBT employer. We welcome and encourage diversity as we strive to create an inclusive workplace."""

bow_lda = id2word.doc2bow(gensim.utils.simple_preprocess(two))
gamma, phi = lda.inference([bow])
two_topics = gamma[0] / gamma[0].sum()
two_topics

In [ ]:
three = """
At Nuance, we empower people with the ability to seamlessly interact with their connected devices and the digital world around them. We are creating a world where technology thinks and acts the way people do by designing the most human, natural, and intuitive ways of interacting with technology.

Our nimble technology uses analytics and advanced algorithms to transform the inanimate into animate and reduce complicated processes into simple ones.

Join our Healthcare team...caring for clinicians the way they care for patients. Beyond words. We create technology that lets clinicians capture and document care quickly and easily so they can focus their attention on their patients.

Join our Mobile team…intelligent systems now ready for the road. We are passionate about developing intelligent interfaces that enable people to talk to their cars, phones, devices, and other smart “things”.

Join our Enterprise team…great customer service starts here. We design virtual assistants for intelligent and effortless customer service helping customers find the information they need using whatever channel they prefer.

Join our Imaging team…greater document flexibility for more personal productivity. We are passionate about designing and building secure technology that empowers companies to gain control of their document management processes and ensure their proprietary and customer information is protected.
Job Summary
Summary: The candidate will pursue cutting edge research aligned with Nuance’s long term AI roadmap. The focus will be on developing or improving deep learning models for Natural Language Understanding. As part of this work, the candidate is expected to publish papers in renowned journals and conferences.

Responsibilities:
Conduct research and experiments to develop or improve deep learning models for Natural Language Understanding.
Implement prototypes to test new ideas and algorithms.
Analyze and improve performance of existing algorithms.
Publish papers in renowned journals and conferences.
Transfer knowledge to teams productizing research ideas.
Perform tasks related to securing and keeping the products, tools, and processes that you are responsible for security.

Qualifications
Number of Years of Work Experience: 2+

Required Skills:
PhD in machine / deep learning or closely related field.
Mastery of machine learning libraries such as TensorFlow or pyTorch.
Good programming and debugging skills; able to prototype research ideas effectively.
Experience running accuracy experiments and systematically improving performance.
Should be able to take initiative, but also follow a plan and work well as part of a team.
Good communication skills (written and oral).

Preferred Skills:
Track record of deep learning publications in renowned journals and conferences.
Good knowledge of the state of the art techniques in Natural Language Understanding, as well as a good understanding of the challenges in this field.
Strong experience in a scripting language (e.g. Python).
Experience working in Linux environment.

Education: PhD in machine / deep learning or closely related field.

Additional Information
Nuance Communication Inc. is an equal opportunity employer. We evaluate qualified applicants without regard to race, color, religion, sex, national origin, disability, veteran status, and other legally protected characteristics. The EEO is the Law poster is available here: http://www.dol.gov/ofccp/regs/compliance/posters/pdf/eeopost.pdf . If you need a reasonable accommodation because of a disability for any part of the employment process, please call 781-565-5000 – Human Resources Department and let us know the nature of your request and your contact information.
"""

bow_lda = id2word.doc2bow(gensim.utils.simple_preprocess(three))
gamma, phi = lda.inference([bow])
three_topics = gamma[0] / gamma[0].sum()
three_topics

In [ ]:
scipy.spatial.distance.pdist([one_topics, two_topics])

In [ ]:
scipy.spatial.distance.pdist([one_topics, three_topics])

In [ ]:
scipy.spatial.distance.pdist([two_topics, three_topics])

In [ ]:
distances = 'braycurtis', 'canberra', 'chebyshev', 'cityblock', 'correlation', 'cosine', 'dice', 'euclidean', 'hamming', 'jaccard', 'jensenshannon', 'kulsinski', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule'

In [ ]:
for d in distances:
    a = scipy.spatial.distance.pdist([one_topics, two_topics], d)
    x = scipy.spatial.distance.pdist([two_topics, three_topics], d)
    y = scipy.spatial.distance.pdist([one_topics, three_topics], d)
    print(d, a / ((x + y)/2))

In [ ]:
import numpy as np

In [ ]:
x = np.asarray([7,8, 9, 0])
idx = np.asarray([], dtype=np.int)
jdx = np.asarray([2,3])

In [ ]:
x[idx]